In [1]:
%reload_ext autoreload
%autoreload 2

# Building spectral library from search engines instead of prediction

Taking alphapept as an example:

### 1. Loading PSMs from alphapept

In [6]:
!pip install alphabase peptdeep

     -------------------------------------- 559.1/559.1 KB 7.0 MB/s eta 0:00:00
     -------------------------------------- 267.7/267.7 KB 8.3 MB/s eta 0:00:00
     ---------------------------------------- 4.7/4.7 MB 17.5 MB/s eta 0:00:00
     -------------------------------------- 161.9/161.9 MB 8.7 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 34.8 MB/s eta 0:00:00
     ---------------------------------------- 120.7/120.7 KB ? eta 0:00:00

You should consider upgrading via the 'C:\Users\admin\.conda\envs\alphapept\python.exe -m pip install --upgrade pip' command.


In [3]:
import os
ap_ms_data_hdf = os.path.expanduser('F:/temp_test_run/thermo_IRT.ms_data.hdf')

In [4]:
from alphabase.io.psm_reader.alphapept_reader import AlphaPeptReader

modification_mapping = {
    'Carbamidomethyl@C': 'cC',
    'Oxidation@M': 'oxM',
    'Phospho@S': 'pS',
    'Phospho@T': 'pT',
    'Phospho@Y': 'pY',
    'Acetyl@Protein N-term': 'a',
}

reader = AlphaPeptReader(modification_mapping=modification_mapping)
reader.import_file(ap_ms_data_hdf)
reader.psm_df

,rt,scan_num,spec_idx,query_id,score,precursor_mz,charge,raw_name,fdr,decoy,sequence,mods,mod_sites,nAA,rt_norm
0,1.711395,650,649,1543,0.901961,509.780765,2,thermo_IRT,0.0,0,GLSATVTGGQK,,,11,0.328026
1,3.800632,2636,2635,1966,0.921569,626.862705,2,thermo_IRT,0.0,0,FLASVSTVLTSK,,,12,0.728473
2,5.217257,3529,3528,2885,1.000000,812.401814,2,thermo_IRT,0.0,0,DVFLGMFLYEYAR,,,13,1.000000
3,5.120342,3469,3468,2894,0.960784,820.398656,2,thermo_IRT,0.0,0,DVFLGMFLYEYAR,Oxidation@M,6,13,0.981424
4,4.969747,3318,3317,2822,0.980392,776.427007,2,thermo_IRT,0.0,0,TRFLEGLSEAVTTK,,,14,0.952559


### 2. Extract fragment intensities from the spectrum file

In [7]:
from peptdeep.rescore.feature_extractor import match_one_raw

psm_df, frag_mz_df, frag_intensity_df, frag_merr_df = match_one_raw(
    reader.psm_df, ms2_file=ap_ms_data_hdf,
    ms2_file_type='alphapept',
    frag_types_to_match=['b_z1','b_z2','y_z1','y_z2'],
    ms2_ppm=True, ms2_tol=50.0,
    calibrate_frag_mass_error=False,
)
psm_df

2022-09-13 22:23:49> Downloading pretrained_models.zip ...
2022-09-13 22:23:52> The pretrained models had been downloaded in C:\Users\admin/peptdeep\pretrained_models\pretrained_models.zip


,rt,scan_num,spec_idx,query_id,score,precursor_mz,charge,raw_name,fdr,decoy,sequence,mods,mod_sites,nAA,rt_norm,frag_start_idx,frag_end_idx
0,1.711395,650,649,1543,0.901961,509.780765,2,thermo_IRT,0.0,0,GLSATVTGGQK,,,11,0.328026,0,10
1,3.800632,2636,2635,1966,0.921569,626.862705,2,thermo_IRT,0.0,0,FLASVSTVLTSK,,,12,0.728473,10,21
2,5.217257,3529,3528,2885,1.000000,812.401814,2,thermo_IRT,0.0,0,DVFLGMFLYEYAR,,,13,1.000000,21,33
3,5.120342,3469,3468,2894,0.960784,820.398656,2,thermo_IRT,0.0,0,DVFLGMFLYEYAR,Oxidation@M,6,13,0.981424,33,45
4,4.969747,3318,3317,2822,0.980392,776.427007,2,thermo_IRT,0.0,0,TRFLEGLSEAVTTK,,,14,0.952559,45,58


#### Normalize the fragment intensities?

In [13]:
# from peptdeep.model.ms2 import normalize_training_intensities

# psm_df.sort_values('frag_start_idx', inplace=True)
# psm_df, frag_intensity_df = normalize_training_intensities(
#     psm_df, frag_intensity_df
# )
# frag_intensity_df

### 3. Generate the library for DiaNN

In [8]:
from peptdeep.protein.fasta import FastaLib

fasta_lib = FastaLib()

In [9]:
fasta_lib._precursor_df = psm_df
fasta_lib._fragment_mz_df = frag_mz_df
fasta_lib._fragment_intensity_df = frag_intensity_df

In [10]:
from peptdeep.spec_lib.translate import translate_to_tsv
translate_to_tsv(
    fasta_lib, 
    tsv = ap_ms_data_hdf[:-len('ms_data.hdf')]+'tsv',
    min_frag_intensity=0.001,
    keep_k_highest_fragments=1000, #keep all matched fragments
)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.21s/it]


Translation finished, it will take several minutes to export the rest precursors to the tsv file...
